# 线性回归
## 3.2 线性回归:简洁实现
### 使用MXNet提供Gluon接口更方便的实现一个线性回归


In [ ]:
# 导包
# matplotlib 作图,嵌入式显示
%matplotlib inline
from IPython import display
from matplotlib import pyplot as plt
from mxnet import autograd,nd
import random


In [ ]:
# 生成数据集

## 样本数量1000,特征数2
num_inputs = 2
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2
## features:1000*2
features = nd.random.normal(scale=1,shape=(num_examples,num_inputs))
## labels:1000*1
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b
labels += nd.random.normal(scale=0.01,shape=labels.shape)

features[0],labels[0]

def use_svg_display():
    # 用矢量图显示
    display.set_matplotlib_formats('svg')

def set_figsize(figsize=(3.5,2.5)):
    use_svg_display()
    # 设置图的尺寸
    plt.rcParams['figure.figsize'] = figsize
    
set_figsize()
plt.scatter(features[:,1].asnumpy(),labels.asnumpy(),1);


In [ ]:
# 读取数据集

# Gluon提供了data包来读取数据

from mxnet.gluon import data as gdata

batch_size = 10
# 将训练的数据集和标签组合
dataset = gdata.ArrayDataset(features,labels)
# 随机读取小批量数据
data_iter = gdata.DataLoader(dataset,batch_size,shuffle=True)

for X,y in data_iter:
    print(X,y)
    break


In [ ]:
# 定义模型

from mxnet.gluon import nn

## Sequential实例可以看做是一个串联各个层的容器
net = nn.Sequential()
# 添加一个输出个数为1的全连接层
net.add(nn.Dense(1))


In [ ]:
# 初始化模型参数

## init提供模型参数初始化的各种方法
from mxnet import init 

## init.Normal(sigma=0.01):均值为0,标准差为0.01的正态分布
net.initialize(init.Normal(sigma=0.01))


In [ ]:
# 定义损失函数

from mxnet.gluon import loss as gloss

## 平方损失又称为L2范数损失
loss = gloss.L2Loss()


In [ ]:
# 定义优化算法

from mxnet import gluon

## 创建一个Trainer实例,并指定学习率为0.03的小批量随机梯度下降(sgd)为优化算法
## net.collect_params():所有通过add函数嵌套的层所包含的全部参数
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.03})


In [ ]:
# 训练模型

## 调用Trainer的step来迭代模型参数
num_epochs = 3

for epoch in range(1,num_epochs + 1): 
    # 在每一个迭代周期中,会使用训练数据集中的所有样本一次(假设样本能被批量大小整除)
    for X,y in data_iter:
        with autograd.record():
            l = loss(net(X),y) # l是有关小批量X和y的损失
        l.backward() # l.backward()等价于l.sum().backward()
        trainer.step(batch_size)
    l = loss(net(features),labels)
    print('epoch %d ,loss %f ' % (epoch,l.mean().asnumpy()))
    


In [ ]:

# 比较学到的参数和用来生成训练集的参数

dense = net[0]
true_w,dense.weight.data()

true_b,dense.bias.data()
